## Web Scrapping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

data_com = []

### Récupération du numéro allociné d'un film

In [47]:
def num_film_allocine(titre_film):
    """permet de trouver le numéro d'un film à partir de son titre"""
    boolean = True
    i = 0
    while boolean:
        url = f'''https://www.allocine.fr/film/fichefilm_gen_cfilm={i}.html'''
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')

        links = soup.findAll("div", {"class": "titlebar titlebar-page"})              

        for link in links:  
            title = link.find("div", {'class':"titlebar-title-lg"}).text
            if str(title.lower()) == titre_film.lower():  
                boolean = False              
                return i
        i += 1 

In [48]:
num_film_allocine('Le Garçon aux cheveux verts')

21

### Web scraping des films Sonic & Inception

In [13]:
def web_scrapping(num_film, note = 5):  
    """recueil tous les commentaires d'un film à partir de son numéro"""
    commentaires= []
    i = 1

    #la boucle se stoppe quand une span particulière est détecté sur une page de commentaire
    while True:
        url = f'''https://www.allocine.fr/film/fichefilm-{num_film}/critiques/spectateurs/?page={i}'''
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')

        links=soup.findAll('div', {"class": "hred review-card cf"})

        i += 1
        
        for link in links: 
            new_com = {}
            new_com['note'] = link.find("span", {'class':"stareval-note"}).text.replace(',', '.')
            new_com['commentaire'] = link.find("div", {'class':"content-txt review-card-content"}).text.replace('"', '').replace("\n", '').replace('’', "'")
            if float(new_com['note']) <= float(note):
                commentaires.append(new_com)

        #permet de stopper le scrapping sans préciser le nombre de page dans la fonction 'web_scrapping'
        stop = soup.find('span', {"class": "button button-md button-primary-full button-right button-disabled"})
        if stop:
            break
    return commentaires

In [20]:
film_com = web_scrapping(53716, 3)
len(film_com)
data_com += film_com
len(data_com)

8036

## Création d'un dataframe de commentaires

In [21]:
df=pd.DataFrame(data_com, columns=['note', 'commentaire'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8036 entries, 0 to 8035
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   note         8036 non-null   object
 1   commentaire  8036 non-null   object
dtypes: object(2)
memory usage: 125.7+ KB


In [22]:
df.to_csv('data_com2.csv', index=False, encoding='utf-8')

In [23]:
empdata = pd.read_csv('data_com2.csv', index_col=False, delimiter = ',')
empdata.head()

,note,commentaire
0,4.0,une bonne suite pour les aventures de sonic au...
1,5.0,Cette suite de sonic est incroyable !Le fan qu...
2,4.5,Très bon film ont retrouve ce qui fait l'espri...
3,4.0,"Super film de ouf, le fait que Knuckles soit d..."
4,3.0,Malgré deux nouveaux personnages de l'univers ...


In [24]:
empdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8036 entries, 0 to 8035
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   note         8036 non-null   float64
 1   commentaire  8034 non-null   object 
dtypes: float64(1), object(1)
memory usage: 125.7+ KB
